In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import json, time
from slugify import slugify
from pathlib import Path

In [2]:
class Issue():
    year = 0
    month = ''
    issue = ''
    pages = {}
    
    def __init__(self, year, month, issue, pages):
        self.year = year
        self.month = month
        self.issue = issue
        self.pages = pages
    
    def as_dict(self):
        return {
            'year': self.year,
            'month': self.month,
            'issue': self.issue,
            'pages': self.pages
        }
    
    def get_slug(self):
        return slugify(issue)
    
    def get_path(self):
        return Path(f'billboards/{self.year}/{self.month}/{self.get_slug()}.json')
    
    def save(self):
        data = self.as_dict()
        json_data = json.dumps(data)
        if not Path(self.get_path()).parent.exists():
            Path(self.get_path()).parent.mkdir(parents=True)
        Path(self.get_path()).write_text(json_data)
        
    def read(self):
        return json.loads(Path(self.get_path()).read_text())
    
    def exists(self):
        return self.get_path().exists()

In [3]:
try:
    b
except:
    b = webdriver.Firefox()

In [4]:
b.get('https://www-proquest-com.proxy.library.nyu.edu/eima/publication/686483?decadeSelected=1980%20-%201989&yearSelected=1933&monthSelected=06&issueNameSelected=01933Y06Y24%2423Jun%2B24%2C%2B1933%243b%2B%2BVol.%2B45%2B%242825%2429')

In [5]:
input('login please')

login please


''

In [77]:
b.get('https://www-proquest-com.proxy.library.nyu.edu/eima/publication/686483?decadeSelected=1980%20-%201989&yearSelected=1933&monthSelected=06&issueNameSelected=01933Y06Y24%2423Jun%2B24%2C%2B1933%243b%2B%2BVol.%2B45%2B%242825%2429')

In [81]:
for year in [1934, 1935, 1936, 1937, 1938, 1939, 1930, 1931, 1932, 1933, 1940]:
    try:
        year_select = Select(b.find_element_by_id('yearSelected'))
        year_elements = {int(x.text): x for x in year_select.options if int(x.text) >= 1930 and int(x.text) <= 1940}
        year_select.select_by_visible_text(year_elements[year].text)
        time.sleep(4)

        for month in ['October', 'November']: # 'December', 'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 
            month_select = Select(b.find_element_by_id('monthSelected'))
            month_options = [x for x in month_select.options]
            month_options.reverse()
            month_elements = {x.text: x for x in month_options}

            month_select.select_by_visible_text(month_elements[month].text)
            time.sleep(4)

            issue_select = Select(b.find_element_by_id('issueSelected'))
            issue_options = [x for x in issue_select.options]
            issue_options.reverse()
            issue_elements = {x.text: x for x in issue_options}

            year_url = b.current_url
            for issue in issue_elements:
                print()
                print(f'--> {month} {year} - {issue}')
                issue_data = Issue(year=year, month=month, issue=issue, pages={})

                if issue_data.exists():
                    print(f'------> already exists')
                    continue

                time.sleep(2)

                issue_select = Select(b.find_element_by_id('issueSelected'))
                issue_select.select_by_visible_text(issue)
                time.sleep(3)
                b.find_element_by_css_selector('input[value="View issue"]').click()
                time.sleep(5)

                try:
                    b.find_element_by_css_selector('a[title="First page"]').click()
                    time.sleep(5)
                except:
                    time.sleep(3)
                    try:
                        b.find_element_by_css_selector('a[title="First page"]').click()
                        time.sleep(5)
                    except:
                        pass

                has_next = True

                page_info = {}

                while has_next:
                    # get results
                    all_results = b.find_elements_by_class_name('resultHeader')
                    unique_pages = {x.find_element_by_class_name('titleAuthorETC').text: x.find_element_by_tag_name('a').get_attribute('href') for x in all_results}

                    page_info = {**page_info, **unique_pages}

                    try:
                        if b.find_element_by_css_selector('a[title="Next page"]'):
                            has_next = True
                        else:
                            has_next = False
                    except:
                        has_next = False

                    if has_next:
                        b.find_element_by_css_selector('a[title="Next page"]').click()
                        time.sleep(10)

                issue_data.pages = page_info
                if issue_data.pages:
                    issue_data.save()
                    print(f'------> saved')
                else:
                    print(f'------> error occurred, issue not saved')
                time.sleep(4)
    except Exception as e:
        print(f"ERROR: {e}")
        time.sleep(20)


--> October 1934 - Oct 6, 1934; Vol. 46 (40)
------> already exists

--> October 1934 - Oct 13, 1934; Vol. 46 (41)
------> already exists

--> October 1934 - Oct 20, 1934; Vol. 46 (42)
------> already exists

--> October 1934 - Oct 27, 1934; Vol. 46 (43)
------> already exists

--> November 1934 - Nov 3, 1934; Vol. 46 (44)
------> already exists

--> November 1934 - Nov 17, 1934; Vol. 46 (46)
------> already exists

--> November 1934 - Nov 24, 1934; Vol. 49 (47)
------> already exists

--> October 1935 - Oct 5, 1935; Vol. 47 (40)
------> already exists

--> October 1935 - Oct 12, 1935; Vol. 47 (41)
------> already exists

--> October 1935 - Oct 26, 1935; Vol. 47 (43)
------> already exists

--> November 1935 - Nov 2, 1935; Vol. 47 (44)
------> already exists

--> November 1935 - Nov 9, 1935; Vol. 47 (45)
------> already exists

--> November 1935 - Nov 16, 1935; Vol. 47 (46)
------> already exists

--> November 1935 - Nov 23, 1935; Vol. 47 (47)
------> already exists

--> November 1935

In [56]:
# Test integrity of Billboard files

for file in Path('billboards').glob("**/**/*.json"):
    data = json.loads(file.read_text())
    if not str(data['year']) in str(file).split('/'):
        print(f"Something is wrong with file {file}")
    if not data['month'] in str(file).split('/'):
        print(f"Something is wrong with file {file}")
    if not file.name == slugify(data['issue'])+'.json':
        print(f"Something is wrong with file {file}")
        
    if len([x.split(':')[-1] for x in data['pages'].keys()]) < 20:
        print(f"Something is likely wrong with file {file}")
    elif len([x.split(':')[-1] for x in data['pages'].keys()]) < 50:
        print(f"Something might be wrong with file {file}")

Something might be wrong with file billboards/1934/April/apr-21-1934-vol-46-16.json
Something might be wrong with file billboards/1931/July/jul-18-1931-vol-43-29.json
